In [1]:
!pip install nerda -q
!pip install seqeval -q
!pip install nerda -q
!pip install flair -q


     |████████████████████████████████| 2.9 MB 10.6 MB/s 
     |████████████████████████████████| 56 kB 2.5 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
     |████████████████████████████████| 596 kB 34.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 319 kB 7.0 MB/s 
     |████████████████████████████████| 19.7 MB 44.3 MB/s 
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 32.0 MB/s 
     |████████████████████████████████| 788 kB 46.8 MB/s 
     |████████████████████████████████| 981 kB 38.4 MB/s 
     |████████████████████████████████| 62 kB 784 kB/s 
ERROR: pip's dependency resolver does not currently take into account al

In [1]:
from NERDA.datasets import get_conll_data, download_conll_data 
from google.colab import files
import pandas as pd
import ast
import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification
from flair.data import Sentence
from flair.models import SequenceTagger
import timeit
from sklearn.model_selection import train_test_split 

uploaded = files.upload()


download_conll_data()
training = get_conll_data('train')
validation = get_conll_data('valid')
testing = get_conll_data('test')

Saving processed_df.csv to processed_df.csv
Saving retrain_processed.csv to retrain_processed.csv
Reading https://data.deepai.org/conll2003.zip


In [2]:
# Download the testb set of the CoNLL-2003 dataset
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2021-10-24 11:07:38--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb’

eng.testb           100%[===================>] 730.56K  --.-KB/s    in 0.04s   

2021-10-24 11:07:38 (20.4 MB/s) - ‘eng.testb’ saved [748096/748096]



In [3]:
tag_scheme = [
'B-PER',
'I-PER',
'B-ORG',
'I-ORG',
'B-LOC',
'I-LOC',
'B-MISC',
'I-MISC'
]

In [4]:
def generate_labels(input_text):
  input_text = str(input_text)
  if input_text.count(" ") > 0:
    if "went to the store" in input_text:
      if input_text.count(" ") > 4:
        return ["B-PER", "I-PER", "O", "O", "O", "O"]
      return ["B-PER", "O", "O", "O", "O"]
    return ["B-PER", "I-PER"]
  else: 
    return ["B-PER"]

In [5]:
transformer = 'studio-ousia/luke-large-finetuned-conll-2003'

# hyperparameters for network
dropout = 0.1

# hyperparameters for training
# training_hyperparameters = {
# 'epochs' : 5,
# 'warmup_steps' : 2500,                                                   
# 'train_batch_size': 2048,                                         
# 'learning_rate': 1e-5
# }

training_hyperparameters = {
'epochs' : 2,
'warmup_steps' : 500,                                                   
'train_batch_size': 13,                                         
'learning_rate': 1e-5
}

In [6]:
def get_sentence_from_name(input_name):
  input_name = str(input_name)
  return input_name.split(" ")

In [7]:
retrain_subset = pd.read_csv("retrain_processed.csv", index_col=0)
retrain_subset["tags_list"] = retrain_subset["Name"].apply(lambda x: generate_labels(x))
retrain_subset["sentences"] = retrain_subset["Name"].apply(lambda x: get_sentence_from_name(x))

rt_train, rt_valid = train_test_split(retrain_subset, test_size=0.15, stratify=retrain_subset['Race']) 

In [8]:
retrain_subset = pd.read_csv("retrain_processed.csv", index_col=0)
retrain_subset["tags_list"] = retrain_subset["Name"].apply(lambda x: generate_labels(x))
retrain_subset["sentences"] = retrain_subset["Name"].apply(lambda x: get_sentence_from_name(x))

In [9]:
retrain_dict = {"sentences": list(rt_train["sentences"]), "tags": list(rt_train["tags_list"])}
valid_dict = {"sentences": list(rt_valid["sentences"]), "tags": list(rt_valid["tags_list"])}

In [10]:
total_sentences = list(retrain_dict["sentences"]) + list(training["sentences"])
total_tags = list(retrain_dict["tags"]) + list(training["tags"])

valid_sentences = list(valid_dict["sentences"]) + list(validation["sentences"])
valid_tags = list(valid_dict["tags"]) + list(validation["tags"])

total_retrain_dict = {"sentences": total_sentences, "tags": total_tags}
total_valid_dict = {"sentences": valid_sentences, "tags": valid_tags}


# LUKE Model Finetuning

In [11]:
# from NERDA.models import NERDA
# model = NERDA(
# dataset_training = total_retrain_dict,
# dataset_validation = total_valid_dict,
# tag_scheme = tag_scheme, 
# tag_outside = 'O',
# transformer = transformer,
# dropout = dropout,
# hyperparameters = training_hyperparameters
# )

In [12]:
# model.train()

# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [23]:
# test = get_conll_data('test')
# model.evaluate_performance(test)

# Custom LUKE Model Testing on Curated Name Dataset


In [14]:
processed_test_df = pd.read_csv("processed_df.csv", index_col=0)
processed_test_df["tags_list"] = processed_test_df["Name"].apply(lambda x: generate_labels(x))
processed_test_df["sentences"] = processed_test_df["Name"].apply(lambda x: get_sentence_from_name(x))
processed_test_dict = {"sentences": list(processed_test_df["sentences"]), "tags": list(processed_test_df["tags_list"])}

In [15]:
# model.evaluate_performance(processed_test_dict)

In [16]:
# curated_test_labels = processed_test_dict["tags"]
# curated_pred_labels = [generate_entities(processed_test_dict["sentences"][i]) for i in range(len(processed_test_dict["sentences"]))]
# # curated_pred_labels = [i[0] for i in curated_pred_labels]

In [17]:
# print(seqeval.metrics.classification_report(curated_test_labels, curated_pred_labels, digits=4)) 

In [18]:
# load tagger
tagger = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2021-10-24 11:08:56,606 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [19]:
def generate_entities(input_string):
  return model.predict([input_string])[0]

In [21]:
processed_white_df = processed_test_df.loc[processed_test_df["Race"]=="White"].reset_index(drop=True)
processed_black_df = processed_test_df.loc[processed_test_df["Race"]=="Black"].reset_index(drop=True)
processed_api_df = processed_test_df.loc[processed_test_df["Race"]=="API"].reset_index(drop=True)
processed_hispanic_df = processed_test_df.loc[processed_test_df["Race"]=="Hispanic"].reset_index(drop=True)

processed_test_dict_w = {"sentences": list(processed_white_df["sentences"]), "tags": list(processed_white_df["tags_list"])}
processed_test_dict_b = {"sentences": list(processed_black_df["sentences"]), "tags": list(processed_black_df["tags_list"])}
processed_test_dict_a = {"sentences": list(processed_api_df["sentences"]), "tags": list(processed_api_df["tags_list"])}
processed_test_dict_h = {"sentences": list(processed_hispanic_df["sentences"]), "tags": list(processed_hispanic_df["tags_list"])}

# Flair Baseline Results

In [195]:
def generate_entities_flair_baseline(input_string):
    output_length = len(input_string)
    input_string = " ".join(input_string)
    sentence = Sentence(input_string)
    # predict NER tags
    tagger.predict(sentence)
    sentence_length = len(sentence)
    values = ["O"] * output_length
    total_string = ""
    tagged_string = sentence.to_tagged_string()

    tagged_dict = sentence.to_dict(tag_type='ner')
    named_entities = tagged_dict["entities"]
    total_entities = []
    total_text = []

    for i in named_entities:
      text = i["text"]
      space_count = text.count(" ")
      entities = []
      current_entity = str(i["labels"][0])[:5]
      current_text = str(i["text"])
      if "ORG" in current_entity:
        current_entity = "ORG"
      if "MISC" in current_entity:
        current_entity = "MISC"
      if "PER" in current_entity:
        current_entity = "PER"
      if "LOC" in current_entity:
        current_entity = "LOC"
      total_text.append(current_text)
      entities.append("B-"+current_entity)
      if space_count >=1: 
        for j in range(space_count):
          entities.append("I-"+current_entity)
      total_entities.append(entities)
    copy_string = input_string

    for i, te in enumerate(total_text):
        copy_string = copy_string.replace(te, str(total_entities[i]).replace(" ", ""), 1)
            
    entity_list = []
    copy_string = copy_string.replace("'].", "']")
    
    for i in copy_string.split(" "):
        prefix = (i[0:4])
        if prefix == "['B-":
          if i[-1] != "]":
            i = i[:i.index("]") + 1]
          entry = [n.strip() for n in ast.literal_eval(i)]
          entity_list.extend(entry)
        else:
            entity_list.append("O")
    return entity_list

In [198]:
def get_named_entities_flair_baseline(input_row, index):
  words = input_row["words"]
  sentence_boundaries = input_row["sentence_boundaries"]
  start = 0
  total_labels = []

  for i in sentence_boundaries: 
    if i != 0: 
      current_string = words[start:i]
      if len(current_string) >= 120:
        midpoint = len(current_string) // 2
        first_half = current_string[:midpoint]
        second_half = current_string[midpoint:]
        prediction = generate_entities_flair_baseline(first_half) + generate_entities_flair_baseline(second_half)
      else: 
        prediction = generate_entities_flair_baseline(current_string)
      total_labels.extend(prediction)
      start = i
  return total_labels

In [199]:
# len(test_documents[149]["labels"])
# # ["labels"][-14]

In [200]:
# for i in range(len(test_labels)):
#   if len(test_labels[i]) != len(pred_labels[i]):
#     print("hit")
#     print(len(test_labels[i]))
#     print(len(pred_labels[i]))
#     print(i)
#     print("-----")

In [202]:
start = timeit.default_timer()
test_labels_flair = [test_documents[i]["labels"] for i in range(len(test_documents))]
pred_labels_flair = [get_named_entities_flair_baseline(test_documents[i], i) for i in range(len(test_documents))]
print(seqeval.metrics.classification_report(test_labels_flair, pred_labels_flair, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

              precision    recall  f1-score   support

         LOC     0.9402    0.8770    0.9075      1666
        MISC     0.8342    0.8759    0.8546       701
         ORG     0.8521    0.9302    0.8894      1647
         PER     0.9621    0.9669    0.9645      1602

   micro avg     0.9049    0.9181    0.9114      5616
   macro avg     0.8971    0.9125    0.9040      5616
weighted avg     0.9074    0.9181    0.9118      5616

Flair Runtime: 101.17649338400042 seconds


In [203]:
start = timeit.default_timer()
curated_test_labels_w = processed_test_dict_w["tags"]
curated_pred_labels_w = [generate_entities_flair_baseline(processed_test_dict_w["sentences"][i]) for i in range(len(processed_test_dict_w["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_w, curated_pred_labels_w, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

KeyboardInterrupt: ignored

In [ ]:
start = timeit.default_timer()
curated_test_labels_b = processed_test_dict_b["tags"]
curated_pred_labels_b = [generate_entities_flair_baseline(processed_test_dict_b["sentences"][i]) for i in range(len(processed_test_dict_b["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_b, curated_pred_labels_b, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

In [ ]:
start = timeit.default_timer()
curated_test_labels_a = processed_test_dict_a["tags"]
curated_pred_labels_a = [generate_entities_flair_baseline(processed_test_dict_a["sentences"][i]) for i in range(len(processed_test_dict_a["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_a, curated_pred_labels_a, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

In [ ]:
start = timeit.default_timer()
curated_test_labels_h = processed_test_dict_h["tags"]
curated_pred_labels_h = [generate_entities_flair_baseline(processed_test_dict_h["sentences"][i]) for i in range(len(processed_test_dict_h["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_h, curated_pred_labels_h, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [24]:
# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))
        
    return documents


def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/877 [00:00<?, ?B/s]

In [192]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

100%|██████████| 231/231 [00:04<00:00, 46.93it/s]


In [ ]:
def get_named_entities_custom_luke(input_row):
  words = input_row["words"]
  sentence_boundaries = input_row["sentence_boundaries"]
  start = 0
  total_labels = []
  for i in sentence_boundaries: 
    if i != 0: 
      current_string = words[start:i]
      if len(current_string) >= 120:
        midpoint = len(current_string) // 2
        first_half = current_string[:midpoint]
        second_half = current_string[midpoint:]
        prediction = model.predict([first_half])[0] + model.predict([second_half])[0]
      else: 
        prediction = model.predict([current_string])[0]
      total_labels.extend(prediction)
      start = i
  return total_labels

In [ ]:
# model.save_network(model_path='model.bin')
# files.download('model.bin') 

In [ ]:
# get_named_entities_flair_baseline([["hello", "I'm", "David"]])

In [ ]:
test_labels = [test_documents[i]["labels"] for i in range(len(test_documents[:50]))]
pred_labels = [get_named_entities_flair_baseline(test_documents[i]) for i in range(len(test_documents[:50]))]
# pred_labels = [get_named_entities_custom_luke(test_documents[i]) for i in range(len(test_documents))]

In [ ]:
len(test_documents[54]["labels"])

In [ ]:
test_documents[13]["words"]

In [ ]:
test_labels[13]

In [ ]:
pred_labels[13]

In [ ]:
[i for i in range(len(pred_labels)) if len(pred_labels[i]) == 31]

In [ ]:
for i, val in enumerate(test_labels): 
  if len(val) != len(pred_labels[i]):
    print(pred_labels[i])
    print(val)
    print(len(test_documents[i]["words"]))
    print(len(val))
    print(len(pred_labels[i]))

    print("-----")

In [ ]:
print(seqeval.metrics.classification_report(test_labels, pred_labels, digits=4)) 

# Baseline LUKE Model Testing on Curated Name Dataset


# Baseline LUKE Model Testing on CoNLL 2003 Dataset
